In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
genius_creds = ''
spotify_client_id = ''
spotify_client_secret = ''

In [83]:
import requests
from urllib.parse import urlencode

In [104]:
import spotipy
import spotipy.util as spotipy_util
scope = 'user-library-read'
username = 'blasternt'
spotify_token = spotipy_util.prompt_for_user_token(username,scope,client_id=spotify_client_id,client_secret=spotify_client_secret,redirect_uri='https://chen-ye.github.io/lyricize/')
if spotify_token:
    spotify = spotipy.Spotify(auth=spotify_token)
else:
    print("Can't get token for", username)

In [105]:
playlist = spotify.user_playlist(username, '0BSdjrucaGOW2uqYLvodj4', fields='tracks,next')

In [106]:
track_items = []
results = playlist['tracks']
while results:
    track_items = track_items + results['items']
    if results['next']:
        results = spotify.next(results)
    else:
        results = None


In [107]:
songs = [(track_result['track']['name'], track_result['track']['artists'][0]['name']) for track_result in track_items]

In [117]:
songs_strings = [title + ',' + artist for (title, artist) in songs]

In [109]:
def search_lyrics_page(query, page=1):
    """Use the web-version of Genius search"""
    endpoint = "search/lyric?"
    params = {'page': page, 'q': query}

    # This endpoint is not part of the API, requires different formatting
    url = "https://genius.com/api/" + endpoint + urlencode(params)
    response = requests.get(url)
    return response.json()['response'] if response else None

In [118]:
def search_lyrics_in_library(query):
    next_page = 1
    while next_page:
        genius_results = search_lyrics(query, next_page)
        genius_lyric_results = genius_results['sections'][0]['hits']
        song_results = [(result['result']['title'], result['result']['primary_artist']['name']) for result in genius_lyric_results]
        print(song_results)
        song_results_filtered = [result for result in genius_lyric_results if (result['result']['title'] + ',' + result['result']['primary_artist']['name'] in songs_strings) ]
        if song_results_filtered:
            print([(result['result']['title'], result['result']['primary_artist']['name'], result) for result in song_results_filtered])
            break
        next_page = genius_results['next_page']

In [ ]:
search_lyrics_in_library("gasping")